# __[Trading strategy and Backtest](https://www.youtube.com/watch?v=5W_Lpz1ZuTI&t=893s)__

## Strategy

- Universe: S&P 500
- Check performance for the last 12 months
- Skip the last month
- Buy winners, Short losers
- Hold portfolio for 1 month

### FAMA

We use six value-weight portfolios formed on size and prior (2-12) returns to construct Mom. The portfolios, which are formed monthly, are the intersections of 2 portfolios formed on size (market equity, ME) and 3 portfolios formed on prior (2-12) return. The monthly size breakpoint is the median NYSE market equity. The monthly prior (2-12) return breakpoints are the 30th and 70th NYSE percentiles.

Mom is the average return on the two high prior return portfolios minus the average return on the two low prior return portfolios,

$$
MOM =  {(Small High + Big High) / 2} - {(Small Low + Big Low) / 2}
$$

The six portfolios used to construct Mom each month include NYSE, AMEX, and NASDAQ stocks with prior return data. To be included in a portfolio for month t (formed at the end of month t-1),

- a stock must have a price for the end of month t-13 and a good return for t-2

In addition, 

- any missing returns from t-12 to t-3 must be -99.0, CRSP's code for a missing price
- Each included stock also must have ME for the end of month t-1


## Workflow

- Retrieve prices for all stocks in ^GSPC
- Measure performance
- Filter winners and losers into deciles
 - *winners* - highest decile
 - *losers* - lowest decile
- Backtest over last 10 years
- Compare with S&P 500 performance

## References

__[Algovibes](https://www.youtube.com/c/Algovibes)__
__[List of S&P 500 companies](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies)__
__[Detail for Monthly Momentum Factor (Mom)](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor.html)__



In [1]:
# essential library packages
import datetime as dt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import yfinance as yf

In [48]:
SnP500     = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0] # 1st table
symbols    = SnP500.Symbol.to_list() # Symbol is 1st non-indexed column in table
tickers    = []
for i in symbols:
    tickers.append(i.replace('.', '-')) # Yahoo Finance uses '-' when '.' is present in original ticker symbol
date_start = dt.date(2009, 12, 31)
date_end   = dt.date(2021, 12, 31)
history    = yf.download(tickers, start=date_start, end=date_end)
prices     = history["Adj Close"]
prices.index = pd.to_datetime(prices.index)

[*********************100%***********************]  503 of 503 completed

1 Failed download:
- CEG: Data doesn't exist for startDate = 1262239200, endDate = 1640930400


In [58]:
monthly_returns = prices.pct_change().resample('M').agg(lambda x: (x + 1).prod() - 1)
past_11_months  = (monthly_returns + 1).rolling(11).apply(np.prod) - 1

In [59]:
formation = dt.datetime(2010, 12, 31)
end_measurement   = formation - MonthEnd(1)
returns_12_months = (past_11_months.loc[end_measurement]).reset_index()
returns_12_months["decile"] = pd.qcut(returns_12_months.iloc[:,1], 10, labels=False, duplicates="drop")

In [75]:
def momentum(formation):
    end_measurement = formation - MonthEnd(1)
    returns_12_months = (past_11_months.loc[end_measurement]).reset_index()
    returns_12_months["decile"] = pd.qcut(returns_12_months.iloc[:,1], 10, labels=False, duplicates="drop")
    winners = returns_12_months[returns_12_months.decile == 9]
    losers  = returns_12_months[returns_12_months.decile == 0]
    winners_returns = monthly_returns.loc[formation + MonthEnd(1), monthly_returns.columns.isin(winners["index"])]
    losers_returns  = monthly_returns.loc[formation + MonthEnd(1), monthly_returns.columns.isin(losers["index"])]
    momentum_profit = winners_returns.mean() - losers_returns.mean()
    return momentum_profit

In [76]:
momentum(formation)

-0.039694055296395275

In [80]:
profits, dates = [], []
for i in range(12 * 10):
    profits.append(momentum(formation + MonthEnd(i)))
    dates.append(formation + MonthEnd(i))

SP = yf.download("^GSPC", start=dates[0], end=dates[-1])
SP = SP["Adj Close"]
SP_monthly = SP.pct_change().resample('M').agg(lambda x: (x + 1).prod() - 1)

[*********************100%***********************]  1 of 1 completed


In [82]:
frame = pd.DataFrame(profits)
frame["S_P500"] = SP_monthly.values
frame["excess"] = frame.iloc[:, 0] - frame.iloc[:, 1]
frame["outperformed"] = ["Yes" if i > 0 else "No" for i in frame.excess]
frame[frame.outperformed == "Yes"].shape[0] / frame.shape[0]

0.39166666666666666